### Bibliotecas

In [13]:
import docker
import os

### Criar Ambiente Docker
_(Caso a imagem já esteja criada no Docker, não é necessário criar novamente.)_

#### Criação Dockerfile

In [2]:
dockerfile_content = """
FROM python:3.12

# Defina o diretório de trabalho
WORKDIR /pysus_extract

# Instalar as dependências
RUN pip install --no-cache-dir pysus==0.15.0 pyarrow

# Comando padrão (opcional, pode ser ajustado conforme a necessidade)
CMD ["python", "script.py"]
"""

with open("Dockerfile", "w") as file:
    file.write(dockerfile_content)

print("Dockerfile criado!")

Dockerfile criado!


#### Criação da Imagem

In [19]:
# Criar cliente Docker
client = docker.from_env()

# Caminho para o diretório que contém o Dockerfile
path = "."

# Construir a imagem Docker com o nome "extract_from_datasus"
image, logs = client.images.build(path=path, tag="extract_from_datasus")

### Extração dos Dados

#### Script da Extração

In [23]:
ano = [2019, 2020, 2021, 2022, 2023, 2024]
mes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
estado = ['SP']
grupo = ['RD']
diagnosticos = ('F31', 'F32', 'F19', 'F2')

In [24]:

settings = f"""
import os
import pandas as pd
from pysus import SIH

output_directory = "/pysus_extract/raw_data"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

ano = {ano}
mes = {mes}
estado = {estado}
grupo = {grupo}
diagnosticos = {diagnosticos}

"""

functions = """
sih = SIH().load() # Loads the files from DATASUS
files = sih.get_files(group=grupo, uf=estado, year=ano, month=mes)
print(files)

def load_data(files, output_diretory):
  for file in files:
    parquet = sih.download(file)
    df = parquet.to_dataframe()
    df = df[df.apply(get_cid, axis=1)]
    file_name = str(file).split('.dbc')[0]
    output_file = os.path.join(output_diretory, f'{file_name}.parquet')
    df.to_parquet(output_file)

def get_cid(lancamento):
    cid_principal = str(lancamento['DIAG_PRINC'])
    if pd.notna(cid_principal) and cid_principal.startswith(diagnosticos):
        return True
    return False
load_data(files, output_directory)
"""

extract_script = settings + functions

with open("script.py", "w") as file:
    file.write(extract_script)

#### Execussão Script no Docker

In [25]:
client = docker.from_env()

path_origin = os.getcwd()

path_output = os.path.join(path_origin, 'raw_data')

# Definir o comando para rodar o container
container = client.containers.run(
    "extract_from_datasus",  # nome da imagem
    "python /pysus_extract/script.py",  # comando que será executado no container
    volumes={
        path_origin: {'bind': '/pysus_extract', 'mode': 'rw'},
        path_output: {'bind': '/pysus_extract/raw_data', 'mode': 'rw'},  # mapeamento do diretório raw_data  # mapeamento de volume
    },
    working_dir='/pysus_extract',  # Diretório de trabalho
    remove=True,  # Remover o container após a execução
)

# Exibir a saída
print(container.decode())


[RDSP1901.dbc, RDSP1902.dbc, RDSP1903.dbc, RDSP1904.dbc, RDSP1905.dbc, RDSP1906.dbc, RDSP1907.dbc, RDSP1908.dbc, RDSP1909.dbc, RDSP1910.dbc, RDSP1911.dbc, RDSP1912.dbc, RDSP2001.dbc, RDSP2002.dbc, RDSP2003.dbc, RDSP2004.dbc, RDSP2005.dbc, RDSP2006.dbc, RDSP2007.dbc, RDSP2008.dbc, RDSP2009.dbc, RDSP2010.dbc, RDSP2011.dbc, RDSP2012.dbc, RDSP2101.dbc, RDSP2102.dbc, RDSP2103.dbc, RDSP2104.dbc, RDSP2105.dbc, RDSP2106.dbc, RDSP2107.dbc, RDSP2108.dbc, RDSP2109.dbc, RDSP2110.dbc, RDSP2111.dbc, RDSP2112.dbc, RDSP2201.dbc, RDSP2202.dbc, RDSP2203.dbc, RDSP2204.dbc, RDSP2205.dbc, RDSP2206.dbc, RDSP2207.dbc, RDSP2208.dbc, RDSP2209.dbc, RDSP2210.dbc, RDSP2211.dbc, RDSP2212.dbc, RDSP2301.dbc, RDSP2302.dbc, RDSP2303.dbc, RDSP2304.dbc, RDSP2305.dbc, RDSP2306.dbc, RDSP2307.dbc, RDSP2308.dbc, RDSP2309.dbc, RDSP2310.dbc, RDSP2311.dbc, RDSP2312.dbc, RDSP2401.dbc, RDSP2402.dbc, RDSP2403.dbc, RDSP2404.dbc, RDSP2405.dbc, RDSP2406.dbc, RDSP2407.dbc, RDSP2408.dbc, RDSP2409.dbc, RDSP2410.dbc, RDSP2411.dbc, RDSP2